In [1]:
metric = 1
#Instrument file Read and fault experiment function
benchmark = ['basicmath/','bitcount/','blackscholes/','fft/','inversek2j/','jmeint/','kmeans/','qsort/','susan/']
e_number = 7
import os
import os.path
root = os.getcwd()
root = os.path.join(root,benchmark[e_number])
class BasicBlock:
  def __init__(self, function, block, begin, end):
    self.function = function
    self.block = block
    self.begin = begin
    self.end =end
     
def faultInject():
  fault_injected = False
  fault_activated = False
  time_out = False
  opt_error = False
  output = "Not injected"
  compile_command = "./compile.sh " + benchmark[e_number]
  os.system(compile_command)
  
  with open(os.path.join(root,"fault/faultInjection")) as fault_file:
    lines = fault_file.readlines()
    for line in lines:
      if line.find("ERROR")!=-1:
        opt_error = True
      elif line.find("Segmation")!=-1:
        opt_error = True
      elif line.find("Aborted")!=-1:
        opt_error = True
      elif line.find("Fault injection has succeeded!")!=-1:
        fault_injected = True 
      elif line.find("The selected instruction is not the supported type")!=-1:
        fault_injected = False
    if opt_error == True:
      output = "OPT error"
      fault_injected = False
      for line in lines:
        print(line)

    if fault_injected == True :
      run_command = "./run.sh " + benchmark[e_number]
      os.system(run_command)
      if os.path.exists(os.path.join(root,"fault/commandLine")) == True:
        with open(os.path.join(root,"fault/commandLine")) as command_file:
            lines = command_file.readlines()
            for line in lines:
              if line == "Fault has been injected!\n":
                fault_activated = True
      else:
        with open(os.path.join(root,'fault/faultOutput'),'r') as command_file:
            lines = command_file.readlines()
            for line in lines:
              if line == "Fault has been injected!\n":
                fault_activated = True
        with open(os.path.join(root,'fault/faultOutput'),'w') as output_file:
            for line in lines:
              if line.strip("\n") != "Fault has been injected!":
                output_file.write(line) 
                    
      with open(os.path.join(root,"fault/stderr")) as stderr_file:
        if fault_activated == True:      
            with open(os.path.join(root,'fault/timeout'),'r') as timeout_file:
              lines = timeout_file.readlines()
              for line in lines:
                if line == "timeout":
                  time_out = True
                  output = "Timeout"
              if time_out == False:
                if os.path.exists(os.path.join(root,'fault/faultOutput')) == True:
                  with open(os.path.join(root,'fault/faultOutput'),'r') as file1:
                    with open(os.path.join(root,'golden_output/output.data'),'r') as file2:
                      fault_output = set(file1)
                      golden_output = set(file2)
                      same = fault_output.intersection(golden_output)
                      if len(same) >= len(golden_output)*metric and len(same)>=len(fault_output)*metric:          
                        output = "Correct"
                      else:
                        output = "SDC"
                else:
                  output = "SDC"
              lines = stderr_file.readlines()
              if len(lines) != 0:
                output = "Detected"    
        else:
          output = "Not activated"
  return output   

instrument_information = []
with open(os.path.join(root,"InstrumentFile")) as instrument_file:
  lines = instrument_file.readlines()
  className = lines[0].split(' ')
  for line in lines[1:]:
    temporary = line.split(' ')
    if len(temporary) != len(className):
      print("Instrument file format error")
      break
    else:
      basicBlock = BasicBlock(temporary[0],temporary[1],temporary[2],temporary[3])
      instrument_information.append(basicBlock)

In [6]:
## Actual experiments process
benchmark_str = benchmark[e_number]+ "experiment beign:"
print(benchmark_str)
with open(os.path.join(root,"experimentResults"),"w") as experiments_result:
  for info in instrument_information:
    sensitive = "N"
    experiments_result.write(info.function+" ")
    experiments_result.write(info.block+" ")
    continue_analyse = True
    print(info.function+" "+info.block+" is being processed")
    for index in range(int(info.begin),int(info.end)+1):
      continue_experiment = True
      if continue_analyse == False:
        break
      for window in range(0,6):
        if continue_experiment == False or continue_analyse == False:
          break
        for bits in range(1,4):
          if window>0 and bits==1:
            continue
          if continue_experiment == False or continue_analyse == False:
            break
          for i in range(1,10):
            if continue_experiment == False or continue_analyse == False:
              break
            with open(os.path.join(root,"faultConfiguration"),"w") as fault_configuration:
              configuration_lines = []
              beginIndex = "beginIndex "+str(index)+"\n"
              windowSize = "windowSize "+str(window)+"\n"
              numofBits = "numofBits "+str(bits)+"\n"
              configuration_lines.append(beginIndex)
              configuration_lines.append(windowSize)
              configuration_lines.append(numofBits)
              fault_configuration.writelines(configuration_lines)
            output = faultInject()
            if output == "Not activated" or output == "Not injected":
              continue_experiment = False
            elif output == "SDC":
              sensitive = "S"
              continue_analyse = False
            elif output == "Opt error":
              print(str(index)+" "+str(window)+" "+str(bits))
    experiments_result.write(sensitive+"\n")    
            
print("Experiments Done!")

qsort/experiment beign:
compare entry is being processed
compare cond.true is being processed
compare cond.false is being processed
compare cond.end is being processed
main entry is being processed
main if.then is being processed
main if.else is being processed
main while.cond is being processed
main land.lhs.true is being processed
main land.lhs.true6 is being processed
main land.rhs is being processed
main land.end is being processed
main while.body is being processed
main while.end is being processed
main if.end is being processed
main for.cond is being processed
main for.body is being processed
main for.inc is being processed
main for.end is being processed
Experiments Done!


In [5]:
benchmark_str = benchmark[e_number]+ "motivational experiment beign:"
print(benchmark_str)
#motivational example
with open(os.path.join(root,"motivationalResults"),"w") as motivational_result:
  for info in instrument_information:
    motivational_result.write(info.function+" ")
    motivational_result.write(info.block+" ")
    motivational_result.write(str(int(info.end)+1-int(info.begin))+" ")                           
    print(info.function+" "+info.block+" is being processed")
    scount = 0.0
    ncount = 0.0
    total = 0.0
    rate = 0.0
    for index in range(int(info.begin),int(info.end)+1):
      sensitive = "N"
      continue_experiment = True
      for window in range(0,6):
        if continue_experiment == False:
          break
        for bits in range(1,4):
          if window>0 and bits==1:
            continue
          if continue_experiment == False:
            break
          with open(os.path.join(root,"faultConfiguration"),"w") as fault_configuration:
            configuration_lines = []
            beginIndex = "beginIndex "+str(index)+"\n"
            windowSize = "windowSize "+str(window)+"\n"
            numofBits = "numofBits "+str(bits)+"\n"
            configuration_lines.append(beginIndex)
            configuration_lines.append(windowSize)
            configuration_lines.append(numofBits)
            fault_configuration.writelines(configuration_lines)
          output = faultInject()
          if output == "Not activated" or output == "Not injected":
            ##print(str(index)+" "+str(window)+" "+str(bits)+" "+output)
            continue_experiment = False
          elif output == "SDC":
            sensitive = "S"
          elif output == "Opt error":
            print(str(index)+" "+str(window)+" "+str(bits))
      if continue_experiment == True:
        total += 1 
        if sensitive == "S":
          scount += 1
        else:
          ncount += 1
    if total>0:
      if scount>=ncount:
        rate = scount/total
      else:
        rate = ncount/total
    motivational_result.write(str(total)+" ") 
    motivational_result.write(str(round(rate,4))+"\n")    
            
print("Experiments Done!")

qsort/motivational experiment beign:
compare entry is being processed
compare cond.true is being processed
compare cond.false is being processed
compare cond.end is being processed
main entry is being processed
main if.then is being processed
main if.else is being processed
main while.cond is being processed
main land.lhs.true is being processed
main land.lhs.true6 is being processed
main land.rhs is being processed
main land.end is being processed
main while.body is being processed
main while.end is being processed
main if.end is being processed
main for.cond is being processed
main for.body is being processed
main for.inc is being processed
main for.end is being processed
Experiments Done!


In [4]:
###fault injection test
with open(os.path.join(root,"faultConfiguration"),"w") as fault_configuration:
            configuration_lines = []
            beginIndex = "beginIndex "+"77"+"\n"
            windowSize = "windowSize "+"0"+"\n"
            numofBits = "numofBits "+"3"+"\n"
            configuration_lines.append(beginIndex)
            configuration_lines.append(windowSize)
            configuration_lines.append(numofBits)
            fault_configuration.writelines(configuration_lines)
output = faultInject()
print(output)

SDC


In [7]:
###excel writer test
import xlsxwriter
benchmark = "blackscholes.xlsx"
workbook = xlsxwriter.Workbook(benchmark)
worksheet = workbook.add_worksheet()

In [8]:
row = 0
col = 0

with open(os.path.join(root,"experimentResults")) as experiment_file:
        with open(os.path.join(root,"motivationalResults")) as motivational_file:
          lines = experiment_file.readlines()
          for line in lines:
            tempory = line.split(" ")
            col = 0 
            for temp in tempory:
              worksheet.write(row,col,temp)
              col = col+1
            row = row+1
          lines = motivational_file.readlines()
          row = 0
          for line in lines:
            tempory = line.split(" ")
            worksheet.write(row,3,tempory[3])
            worksheet.write(row,4,tempory[4])
            row = row+1
workbook.close()
print("Excel file written!")

Excel file written!


In [12]:
##instruction rate in motivation
rate_90 = 0.0
rate_70 = 0.0
rate_50 = 0.0
min_rate = 0.0
total = 0.0
with open(os.path.join(root,"motivationalResults")) as motivational_file:
          lines = motivational_file.readlines()
          for line in lines:
            tempory = line.split(" ")
            insert = float(tempory[3])
            if insert > 0:
              total = total + 1
              rate = float(tempory[4])
              if rate>0.9:
                rate_90 = rate_90 + 1
              elif rate>0.7:
                rate_70 = rate_70 + 1
              elif rate>0.5:
                rate_50 = rate_50 + 1
              else:
                min_rate = min_rate + 1
print(">90%: "+str(round(rate_90/total,4)))
print("70%~90%: "+str(round(rate_70/total,4)))
print("50~70%: "+str(round(rate_50/total,4)))
print("<50: "+str(round(min_rate/total,4)))

>90%: 0.8261
70%~90%: 0.0435
50~70%: 0.1304
<50: 0.0


In [11]:
import os
import os.path
a = []
total = 0
root = os.getcwd()
with open(os.path.join(root,"motivationalResults")) as motivational_file:
          lines = motivational_file.readlines()
          for line in lines:
            tempory = line.split(" ")
            insert = float(tempory[3])
            if insert > 0:
              a.append(insert)
              total = total + insert
sort = sorted(a)
print(sort)
index = int(len(sort)/2)
min_skip = 0
max_skip = 0
for temp in sort[:index]:
  min_skip = min_skip + temp
for temp in sort[len(sort)-index:]:
  max_skip = max_skip + temp 
min_speedup = total-min_skip
max_speedup = total-max_skip
print(max_speedup)
print(min_speedup)
print(total)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 6.0, 6.0, 9.0, 16.0, 25.0, 27.0, 37.0, 42.0, 43.0]
228.0
36.0
235.0
